In [ ]:
import torch

import pandas as pd

from tqdm import tqdm
from model import CryptoTransformer, MyDataset
from utils import weighted_corr

In [ ]:
processed_test = pd.read_csv('./processed_data/processed_test.gz')
asset_list = pd.read_csv('./g-research-crypto-forecasting/asset_details.csv')

In [ ]:
net = torch.load('./checkpoint/model_v1.0/step_1.pt')
test_dataset = MyDataset(processed_test)

In [ ]:
predictions = []
net.eval()
net = net.cpu()
with torch.no_grad():
    with tqdm(test_dataset) as t:
        for x, y, num_assets in t:
            y_pred = net(x)
            y_pred = y_pred[:num_assets]
            predictions.append(y_pred)

In [ ]:
predictions = torch.cat(predictions).numpy()
processed_test['Predictions'] = predictions
asset_list.set_index('Asset_ID', inplace=True)
processed_test['Weight'] = processed_test['Asset_ID'].apply(lambda x: asset_list.loc[x]['Weight'])

In [ ]:
corr = weighted_corr(processed_test['Predictions'], processed_test['Target'], processed_test['Weight'])
print(f'Weighted Pearson Coefficient: {corr:.2e}')